##Spark DataFrames

###Imports

For these examples, we just need to import two **pyspark.sql** libraries:
- `types`
- `functions`

We need `pyspark.sql.types` to define schemas for the DataFrames. The `pyspark.sql.functions` library contains all of the functions specific to SQL and DataFrames in **PySpark**.

In [0]:
from pyspark.sql.types import *  # Necessary for creating schemas
from pyspark.sql.functions import * # Importing PySpark functions

###Creating DataFrames

#### Making a Simple DataFrame from a Tuple List

In [0]:
# Make a tuple list
a_list = [('a', 1), ('b', 2), ('c', 3)]

# Create a Spark DataFrame, without supplying a schema value
df_from_list_no_schema = \
sqlContext.createDataFrame(a_list)

# Print the DF object
print(df_from_list_no_schema)

# Print a collected list of Row objects
print(df_from_list_no_schema.collect())

# Show the DataFrame
df_from_list_no_schema.show()

DataFrame[_1: string, _2: bigint]
[Row(_1='a', _2=1), Row(_1='b', _2=2), Row(_1='c', _2=3)]
+---+---+
 _1| _2|
+---+---+
 a| 1|
 b| 2|
 c| 3|
+---+---+

#### Making a Simple DataFrame from a Tuple List and a Schema

In [0]:
# Create a Spark DataFrame, this time with schema
df_from_list_with_schema = \
sqlContext.createDataFrame(a_list, ['letters', 'numbers']) # this simple schema contains just column names

# Show the DataFrame
df_from_list_with_schema.show()

# Show the DataFrame's schema
df_from_list_with_schema.printSchema()

+-------+-------+
letters|numbers|
+-------+-------+
 a| 1|
 b| 2|
 c| 3|
+-------+-------+

root
-- letters: string (nullable = true)
-- numbers: long (nullable = true)

#### Making a Simple DataFrame from a Dictionary

In [0]:
# Make a dictionary
a_dict = [{'letters': 'a', 'numbers': 1},
          {'letters': 'b', 'numbers': 2},
          {'letters': 'c', 'numbers': 3}]

# Create a Spark DataFrame from the dictionary
df_from_dict = \
(sqlContext
 .createDataFrame(a_dict)) # You will get a warning about this

# Show the DataFrame
df_from_dict.show()

/databricks/spark/python/pyspark/sql/session.py:355: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
 warnings.warn("inferring schema from dict is deprecated,"
+-------+-------+
letters|numbers|
+-------+-------+
 a| 1|
 b| 2|
 c| 3|
+-------+-------+

#### Making a Simple DataFrame Using a StructType Schema + RDD

In [0]:
# Define the schema
schema = StructType([
    StructField('letters', StringType(), True),
    StructField('numbers', IntegerType(), True)])

# Create an RDD from a list
rdd = sc.parallelize(a_list)

# Create the DataFrame from these raw components
nice_df = \
(sqlContext
 .createDataFrame(rdd, schema))

# Show the DataFrame
nice_df.show()

+-------+-------+
letters|numbers|
+-------+-------+
 a| 1|
 b| 2|
 c| 3|
+-------+-------+

In [0]:
# Define the schema
schema = StructType([
    StructField('letters', StringType(), True),
    StructField('numbers', IntegerType(), True)])

# Create an RDD from a list
rdd = sc.parallelize(a_list)

# Create the DataFrame from these raw components
nice_df = \
(sqlContext
 .createDataFrame(rdd, schema))

# Show the DataFrame
nice_df.show()

+-------+-------+
letters|numbers|
+-------+-------+
 a| 1|
 b| 2|
 c| 3|
+-------+-------+

###Simple Inspection Functions

We now have a `nice_df`, here are some nice functions for inspecting the DataFrame.

In [0]:
# `columns`: return all column names as a list
nice_df.columns

Out[8]: ['letters', 'numbers']

In [0]:
# `dtypes`: get the datatypes for all columns
nice_df.dtypes

Out[9]: [('letters', 'string'), ('numbers', 'int')]

In [0]:
# `printSchema()`: prints the schema of the supplied DF
nice_df.printSchema()

root
-- letters: string (nullable = true)
-- numbers: integer (nullable = true)

In [0]:
# `schema`: returns the schema of the provided DF as `StructType` schema
nice_df.schema

Out[11]: StructType(List(StructField(letters,StringType,true),StructField(numbers,IntegerType,true)))

In [0]:
# `first()` returns the first row as a Row while
# `head()` and `take()` return `n` number of Row objects
print(nice_df.first()) # can't supply a value; never a list
print(nice_df.head(2)) # can optionally supply a value (default: 1);
                      # with n > 1, a list
print(nice_df.take(2)) # expects a value; always a list

Row(letters='a', numbers=1)
[Row(letters='a', numbers=1), Row(letters='b', numbers=2)]
[Row(letters='a', numbers=1), Row(letters='b', numbers=2)]

In [0]:
# `count()`: returns a count of all rows in DF
nice_df.count()

Out[14]: 3

In [0]:
# `describe()`: print out stats for numerical columns
nice_df.describe().show() # can optionally supply a list of column names

+-------+-------+-------+
summary|letters|numbers|
+-------+-------+-------+
 count| 3| 3|
 mean| null| 2.0|
 stddev| null| 1.0|
 min| a| 1|
 max| c| 3|
+-------+-------+-------+

In [0]:
# the `explain()` function explains the under-the-hood evaluation process
nice_df.explain()

== Physical Plan ==
*(1) Scan ExistingRDD[letters#70,numbers#71]

###Relatively Simple DataFrame Manipulation Functions

Let's use these functions:
- `unionAll()`: combine two DataFrames together
- `orderBy()`: perform sorting of DataFrame columns
- `select()`: select which DataFrame columns to retain
- `drop()`: select a single DataFrame column to remove
- `filter()`: retain DataFrame rows that match a condition

In [0]:
(nice_df
 .unionAll(nice_df)
 .show())

+-------+-------+
letters|numbers|
+-------+-------+
 a| 1|
 b| 2|
 c| 3|
 a| 1|
 b| 2|
 c| 3|
+-------+-------+

In [0]:
# Add it to itself twice
(nice_df
 .unionAll(nice_df)
 .unionAll(nice_df)
 .show())



+-------+-------+
letters|numbers|
+-------+-------+
 a| 1|
 b| 2|
 c| 3|
 a| 1|
 b| 2|
 c| 3|
 a| 1|
 b| 2|
 c| 3|
+-------+-------+

In [0]:
# Sorting the DataFrame by the `numbers` column
(nice_df
 .unionAll(nice_df)
 .unionAll(nice_df)
 .orderBy('numbers')
 .show())

# Sort the same column in reverse order
(nice_df
 .unionAll(nice_df)
 .unionAll(nice_df)
 .orderBy('numbers',
          ascending = False)
 .show())

+-------+-------+
letters|numbers|
+-------+-------+
 a| 1|
 a| 1|
 a| 1|
 b| 2|
 b| 2|
 b| 2|
 c| 3|
 c| 3|
 c| 3|
+-------+-------+

+-------+-------+
letters|numbers|
+-------+-------+
 c| 3|
 c| 3|
 c| 3|
 b| 2|
 b| 2|
 b| 2|
 a| 1|
 a| 1|
 a| 1|
+-------+-------+

In [0]:
# `select()` and `drop()` both take a list of column names
# and these functions do exactly what you might expect

# Select only the first column of the DF
(nice_df
 .select('letters')
 .show())

# Re-order columns in the DF using `select()`
(nice_df
 .select(['numbers', 'letters'])
 .show())

# Drop the second column of the DF
(nice_df
 .drop('letters')
 .show())

+-------+
letters|
+-------+
 a|
 b|
 c|
+-------+

+-------+-------+
numbers|letters|
+-------+-------+
 1| a|
 2| b|
 3| c|
+-------+-------+

+-------+
numbers|
+-------+
 1|
 2|
 3|
+-------+

In [0]:
# The `filter()` function performs filtering of DF rows

# Here is some numeric filtering with comparison operators
# (>, <, >=, <=, ==, != all work)

# Filter rows where values in `numbers` is > 1
(nice_df
 .filter(nice_df.numbers > 1)
 .show())

# Perform two filter operations
(nice_df
 .filter(nice_df.numbers > 1)
 .filter(nice_df.numbers < 3)
 .show())

# Not just numbers! Use the `filter()` + `isin()`
# combo to filter on string columns with a set of values
(nice_df
 .filter(nice_df.letters
         .isin(['a', 'b']))
 .show())

+-------+-------+
letters|numbers|
+-------+-------+
 b| 2|
 c| 3|
+-------+-------+

+-------+-------+
letters|numbers|
+-------+-------+
 b| 2|
+-------+-------+

+-------+-------+
letters|numbers|
+-------+-------+
 a| 1|
 b| 2|
+-------+-------+